In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
print("User:", os.getenv("METEO_USERNAME"))
print("Pass:", os.getenv("METEO_PASSWORD"))



User: uRN4G7T08eD0aXWdqOjn
Pass: sarnecki_marcel


In [1]:
import geopandas as gpd
gdf = gpd.read_file("data/shape/ne_10m_admin_0_countries.shp")
print(gdf.crs)
print(gdf.geometry.geom_type.value_counts())
print(gdf.head())


EPSG:4326
MultiPolygon    151
Polygon         107
Name: count, dtype: int64
        featurecla  scalerank  LABELRANK SOVEREIGNT SOV_A3  ADM0_DIF  LEVEL  \
0  Admin-0 country          0          2  Indonesia    IDN         0      2   
1  Admin-0 country          0          3   Malaysia    MYS         0      2   
2  Admin-0 country          0          2      Chile    CHL         0      2   
3  Admin-0 country          0          3    Bolivia    BOL         0      2   
4  Admin-0 country          0          2       Peru    PER         0      2   

                TYPE TLC      ADMIN  ... FCLASS_TR  FCLASS_ID FCLASS_PL  \
0  Sovereign country   1  Indonesia  ...      None       None      None   
1  Sovereign country   1   Malaysia  ...      None       None      None   
2  Sovereign country   1      Chile  ...      None       None      None   
3  Sovereign country   1    Bolivia  ...      None       None      None   
4  Sovereign country   1       Peru  ...      None       None      None   

In [2]:
import geopandas as gpd

gdf = gpd.read_file("data/shape/ne_10m_admin_0_countries.shp")
gdf.to_file("data/shape/ne_10m_admin_0_countries.geojson", driver="GeoJSON")

In [1]:
import geopandas as gpd
gdf = gpd.read_file("data/shape/ne_10m_admin_0_countries.geojson")
print(gdf.geometry.geom_type.value_counts())


MultiPolygon    151
Polygon         107
Name: count, dtype: int64


In [1]:
import geopandas as gpd

subs = gpd.read_file("data/osm/substations_110kV_clean.geojson")
lines = gpd.read_file("data/osm/lines_110kV_clean.geojson")

print(subs.crs)
print(lines.crs)


C:\Users\Marcel\.conda\envs\parade\lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: Several features with id = 27144164 have been found. Altering it to be unique. This warning will not be emitted anymore for this layer
  return ogr_read(
C:\Users\Marcel\.conda\envs\parade\lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: Several features with id = 66493710 have been found. Altering it to be unique. This warning will not be emitted anymore for this layer
  return ogr_read(


EPSG:4326
EPSG:4326


In [2]:
import geopandas as gpd
lines = gpd.read_file("data/osm/lines_110kV_clean.geojson")
print(len(lines), lines.total_bounds)

1470 [-9.6930478 51.6756821 -6.0818068 55.1316259]


C:\Users\Marcel\.conda\envs\parade\lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: Several features with id = 66493710 have been found. Altering it to be unique. This warning will not be emitted anymore for this layer
  return ogr_read(


In [12]:
import pandas as pd
import os

# --- File path ---
INPUT_PATH = r"C:\Users\Marcel\will_it_rain_on_my_parade\data\raw\System_Data_Qtr_Hourly_2024(1).xlsx"
OUTPUT_PATH = "data/processed/ireland_demand_hourly_2024.csv"

# --- Load Excel ---
# Adjust sheet_name if needed (e.g., "Sheet1" or "Demand Data")
df = pd.read_excel(INPUT_PATH)

# --- Normalize column names (remove spaces etc.) ---
df.columns = df.columns.str.strip()

# --- Parse timestamp ---
df["DateTime"] = pd.to_datetime(df["DateTime"], errors="coerce")

# --- Keep only relevant columns ---
if "IE Demand" not in df.columns:
    raise ValueError("❌ 'IE Demand' column not found in Excel sheet")

df = df[["DateTime", "IE Demand"]].dropna()

# --- Convert demand to numeric ---
df["IE Demand"] = pd.to_numeric(df["IE Demand"], errors="coerce")

# --- Aggregate to hourly average ---
df_hourly = (
    df.set_index("DateTime")
      .resample("1H")
      .mean()
      .reset_index()
)

# --- Rename for clarity ---
df_hourly.rename(columns={"IE Demand": "system_demand_MW"}, inplace=True)

# --- Save cleaned result ---
os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)
df_hourly.to_csv(OUTPUT_PATH, index=False)

print("✅ Saved hourly Ireland demand →", OUTPUT_PATH)
print(df_hourly.head())


✅ Saved hourly Ireland demand → data/processed/ireland_demand_hourly_2024.csv
             DateTime  system_demand_MW
0 2024-01-01 00:00:00         3412.3650
1 2024-01-01 01:00:00         3264.4625
2 2024-01-01 02:00:00         3147.2175
3 2024-01-01 03:00:00         3017.0750
4 2024-01-01 04:00:00         2934.5750


C:\Users\Marcel\AppData\Local\Temp\ipykernel_24148\1437612487.py:30: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  .resample("1H")


In [ ]:
import os
import geopandas as gpd
from shapely.geometry import box

# Define project root relative to this script
PROJECT_ROOT = os.path.abspath(r"C:\Users\Marcel\will_it_rain_on_my_parade")

# Paths
BUILT_PATH = os.path.join(PROJECT_ROOT, "data", "shape", "ne_10m_urban_areas.shp")
COUNTRIES_PATH = os.path.join(PROJECT_ROOT, "data", "shape", "ne_10m_admin_0_countries.geojson")

# Load Ireland boundary
countries = gpd.read_file(COUNTRIES_PATH)
ireland = countries[
    (countries["ADMIN"].str.lower() == "ireland") |
    (countries["NAME_LONG"].str.lower() == "ireland")
].to_crs("EPSG:4326")

# Load built-up areas (global)
built = gpd.read_file(BUILT_PATH)

# Clip to Ireland
built_ie = gpd.clip(built.to_crs("EPSG:4326"), ireland)

# Save filtered file
out_path = os.path.join(PROJECT_ROOT, "data", "shape", "built_up_ireland.geojson")
built_ie.to_file(out_path, driver="GeoJSON")

print(f"✅ Saved Ireland-only built-up areas → {out_path} ({len(built_ie)} features)")



In [19]:
import geopandas as gpd
import folium

# Load your Ireland-only built-up GeoJSON
geo_path = r"C:\Users\Marcel\will_it_rain_on_my_parade\data\shape\built_up_ireland.geojson"
built_ie = gpd.read_file(geo_path)

# ✅ Fallback name column (since Natural Earth urban areas have no 'name' field)
if "name" not in built_ie.columns:
    built_ie["name"] = built_ie.apply(
        lambda r: f"Urban area #{r.name} ({round(r.get('area_sqkm', 0), 1)} km²)", axis=1
    )

# Compute approximate center for map view
bounds = built_ie.total_bounds
center_lat = (bounds[1] + bounds[3]) / 2
center_lon = (bounds[0] + bounds[2]) / 2

# Create Folium map
m = folium.Map(location=[center_lat, center_lon], zoom_start=7, tiles="cartodbpositron")

# Add built-up areas (gray polygons)
folium.GeoJson(
    built_ie.to_json(),
    name="🏙️ Built-up Areas (Ireland)",
    style_function=lambda x: {
        "color": "#666666",
        "fillColor": "#999999",
        "fillOpacity": 0.5,
        "weight": 0.8,
    },
    tooltip=folium.GeoJsonTooltip(fields=["name"], aliases=["Urban Area:"]),
).add_to(m)

folium.LayerControl(collapsed=False).add_to(m)

# Save temporary HTML preview
m.save("preview_built_up_ireland.html")

print("✅ Map preview generated → preview_built_up_ireland.html")
print("🌍 Open it in your browser to explore.")


✅ Map preview generated → preview_built_up_ireland.html
🌍 Open it in your browser to explore.


In [ ]:
import pystac_client
catalog = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")
for coll in catalog.get_collections():
    print(coll.id)

In [30]:
import xarray as xr

ds = xr.open_dataset("C:/Users/Marcel/will_it_rain_on_my_parade/data/era5/era5_ireland_mean_wind_2009_2024.nc")
CLIM_PATH = os.path.join(PROJECT_ROOT, "data", "era5", "era5_ireland_mean_wind_2009_2024.csv")

print(ds)



<xarray.Dataset> Size: 2kB
Dimensions:                        (lon: 23, lat: 17)
Coordinates:
  * lon                            (lon) float32 92B 349.0 349.2 ... 354.2 354.5
  * lat                            (lat) float32 68B 55.5 55.25 ... 51.75 51.5
Data variables:
    __xarray_dataarray_variable__  (lat, lon) float32 2kB ...
    spatial_ref                    int32 4B ...


In [28]:
import os5_ireland_mean_wind_2009_2024.csv

for root, dirs, files in os.walk("C:/Users/Marcel/will_it_rain_on_my_parade"):
    for f in files:
        if f.endswith(".nc"):
            print(os.path.join(root, f))



C:/Users/Marcel/will_it_rain_on_my_parade\data\era5\era5_ie_2020.nc
C:/Users/Marcel/will_it_rain_on_my_parade\src\utils\data\era5\era5_ireland_mean_wind_2009_2024.nc


In [32]:
import xarray as xr
import pandas as pd
import os

# === Corrected paths ===
PROJECT_ROOT = r"C:\Users\Marcel\will_it_rain_on_my_parade"
NC_PATH = os.path.join(PROJECT_ROOT, "data", "era5", "era5_ireland_mean_wind_2009_2024.nc")
CSV_OUT = os.path.join(PROJECT_ROOT, "data", "era5", "era5_ireland_mean_wind_1994_2024.csv")

# === Load dataset ===
print(f"📂 Loading NetCDF: {NC_PATH}")
ds = xr.open_dataset(NC_PATH)

# === Identify variable ===
varname = list(ds.data_vars.keys())[0]
print(f"📊 Found variable: {varname}")

# === Convert to DataFrame ===
df = ds[varname].to_dataframe(name="wind_speed_10m").reset_index()

# === Clean and save ===
df = df.rename(columns={"lat": "latitude", "lon": "longitude"})
os.makedirs(os.path.dirname(CSV_OUT), exist_ok=True)
df.to_csv(CSV_OUT, index=False)

print(f"✅ Exported CSV → {CSV_OUT}")
print(df.head())



📂 Loading NetCDF: C:\Users\Marcel\will_it_rain_on_my_parade\data\era5\era5_ireland_mean_wind_2009_2024.nc
📊 Found variable: __xarray_dataarray_variable__
✅ Exported CSV → C:\Users\Marcel\will_it_rain_on_my_parade\data\era5\era5_ireland_mean_wind_1994_2024.csv
   latitude  longitude  wind_speed_10m
0      55.5     349.00        9.233665
1      55.5     349.25        9.200921
2      55.5     349.50        9.173604
3      55.5     349.75        9.095283
4      55.5     350.00        9.049292


In [33]:
import pystac_client

catalog = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1/")
search = catalog.search(
    collections=["era5-pds"],
    datetime="2019-01-01/2019-12-31",
    query={"era5:kind": {"eq": "an"}}
)

items = list(search.item_collection())
print(len(items))
for item in items[:3]:
    print(item.id)



12
era5-pds-2019-12-an
era5-pds-2019-11-an
era5-pds-2019-10-an


In [1]:
import rasterio, numpy as np, os

tif_path = r"C:\Users\Marcel\will_it_rain_on_my_parade\data\era5\era5_ireland_std_1994_2024.tif"

with rasterio.open(tif_path) as src:
    arr = src.read(1)
    print("Shape:", arr.shape)
    print("Min:", np.nanmin(arr), "Max:", np.nanmax(arr))
    print("NaN count:", np.isnan(arr).sum(), "of", arr.size)

Shape: (17, 23)
Min: 1.438983 Max: 3.5367448
NaN count: 0 of 391


In [1]:
import rasterio
import numpy as np

path = "data/era5/era5_ireland_mean_1994_2024.tif"
with rasterio.open(path) as src:
    arr = src.read(1)
    print("Mean:", np.nanmean(arr))
    print("Min :", np.nanmin(arr))
    print("Max :", np.nanmax(arr))

Mean: 6.25933
Min : 3.551474
Max : 9.192362
